# 資料合併-長程

In [6]:
import os
import pandas as pd

def merge_files(file_paths, output_path):
    data_frames = []
    
    # 逐一讀取檔案
    for file_path in file_paths:
        if os.path.exists(file_path):
            print(f"正在讀取檔案：{file_path}")
            df = pd.read_csv(file_path)
            if not df.empty:
                data_frames.append(df)
            else:
                print(f"檔案 {file_path} 為空，跳過。")
        else:
            print(f"檔案 {file_path} 不存在，跳過。")

    # 合併所有資料
    if data_frames:
        merged_data = pd.concat(data_frames, ignore_index=True)
        merged_data.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"合併完成，結果儲存至：{output_path}")
    else:
        print("沒有任何資料可供合併。")

# 主程式
base_path = '/Users/yuchingchen/Documents/專題/cleaned_data/data/long'
output_path = '/Users/yuchingchen/Documents/專題/cleaned_data/long_flight.csv'

# 檔案名稱清單
file_paths = [
    f'{base_path}/sydney.csv',
    f'{base_path}/zurich.csv',
    f'{base_path}/losangeles.csv',
    f'{base_path}/frankfurt.csv',
    f'{base_path}/france.csv',
    f'{base_path}/newyork.csv',
    f'{base_path}/london.csv',
]

# 合併檔案
merge_files(file_paths, output_path)

正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/sydney.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/zurich.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/losangeles.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/frankfurt.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/france.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/newyork.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/london.csv
合併完成，結果儲存至：/Users/yuchingchen/Documents/專題/cleaned_data/long_flight.csv


# 資料合併-長程-商務艙

In [ ]:
import os
import pandas as pd

def merge_files(file_paths, output_path):
    data_frames = []
    
    # 逐一讀取檔案
    for file_path in file_paths:
        if os.path.exists(file_path):
            print(f"正在讀取檔案：{file_path}")
            df = pd.read_csv(file_path)
            if not df.empty:
                data_frames.append(df)
            else:
                print(f"檔案 {file_path} 為空，跳過。")
        else:
            print(f"檔案 {file_path} 不存在，跳過。")

    # 合併所有資料
    if data_frames:
        merged_data = pd.concat(data_frames, ignore_index=True)
        merged_data.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"合併完成，結果儲存至：{output_path}")
    else:
        print("沒有任何資料可供合併。")

# 主程式
base_path = '/Users/yuchingchen/Documents/專題/cleaned_data/data/long'
output_path = '/Users/yuchingchen/Documents/專題/cleaned_data/long_flight_bsuiness.csv'

# 檔案名稱清單
file_paths = [
    f'{base_path}/sydney_business.csv',
    f'{base_path}/zurich_business.csv',
    f'{base_path}/losangeles_business.csv',
    f'{base_path}/frankfurt_business.csv',
    f'{base_path}/france_business.csv',
    f'{base_path}/newyork_business.csv',
    f'{base_path}/london_business.csv',
]

# 合併檔案
merge_files(file_paths, output_path)

# 長程航班-經濟艙

In [9]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

# --------------------------
# 1. 載入資料及初步處理
# --------------------------
data_path = '/Users/yuchingchen/Documents/專題/cleaned_data/long_flight.csv'
data = pd.read_csv(data_path, low_memory=False)

# 統一「出發日期」格式為 "YYYY-MM-DD"
data["出發日期"] = pd.to_datetime(
    data["出發日期"], 
    format="%Y-%m-%d", 
    errors="coerce"
).combine_first(
    pd.to_datetime(data["出發日期"], format="%Y/%m/%d", errors="coerce")
)
data["出發日期"] = data["出發日期"].dt.strftime("%Y-%m-%d")

# --------------------------
# 2. 計算對數變數與新增最低價格天數
# --------------------------
data["平均價格_log"] = data["平均價格"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan)
data["最低價格_log"] = data["最低價格"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan)
data["價格變異_log"] = data["價格變異"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan)
data["中位數價格_log"] = data["中位數價格"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan)

# 新增最低價格天數 (91 - '最低價格剩餘天數')
data['最低價格天數'] = 91 - data['最低價格剩餘天數']

# 移除關鍵欄位有 NaN 的資料
data = data.dropna(subset=["平均價格_log", "最低價格_log", "最低價格天數", "價格變異_log", "中位數價格_log"])

# 顯示抵達機場代號種類及數量
print(data['抵達機場代號'].value_counts())

# --------------------------
# 3. 假期判斷
# --------------------------
# 定義各國假期範圍
holidays = {
    "台灣": [("2025-01-25", "2025-02-02"), ("2025-02-28", "2025-03-02")],
    "美國": [("2025-01-18", "2025-01-20"), ("2025-02-15", "2025-02-17")],
    "澳洲": [("2025-01-25", "2025-01-27")],
    "瑞士": [("2025-01-29", "2025-01-30")]
}

# 抵達機場代號與地區對應
airport_to_region = {
    "LAX": "美國",  # 洛杉磯
    "SYD": "澳洲",  # 雪梨
    "FRA": "德國",  # 法蘭克福
    "CDG": "法國",  # 巴黎
    "JFK": "美國",  # 紐約
    "LHR": "英國",  # 倫敦
    "ZRH": "瑞士"   # 蘇黎世
}

# 轉換「出發日期」為 datetime 格式（方便比較）
data["出發日期"] = pd.to_datetime(data["出發日期"])

def is_holiday(row):
    # 台灣假期（不論目的地，出發地為台灣）
    for start, end in holidays["台灣"]:
        if pd.to_datetime(start) <= row["出發日期"] <= pd.to_datetime(end):
            return 1
    # 依據抵達機場對應的地區判斷其他國家假期
    region = airport_to_region.get(row["抵達機場代號"])
    if region and region in holidays:
        for start, end in holidays[region]:
            if pd.to_datetime(start) <= row["出發日期"] <= pd.to_datetime(end):
                return 1
    return 0

data["假期"] = data.apply(is_holiday, axis=1)

# 將「假期」欄位移至「艙等」欄位之後（若存在「艙等」欄位）
cols = data.columns.tolist()
if '假期' in cols and '艙等' in cols:
    cols.insert(cols.index('艙等') + 1, cols.pop(cols.index('假期')))
data = data[cols]

# --------------------------
# 4. 合併額外地區經濟資料
# --------------------------
additional_data = {
    "Region": ["美國", "澳洲", "德國", "英國", "法國", "瑞士"],
    "Cost of Living Index": [70.4, 70.2, 62.2, 62.0, 63.7, 101.1],
    "GDP (PPP) per capita (in thousand USD)": [82.715, 67.901, 69.532, 60.735, 63.881, 93.055],
}
region_df = pd.DataFrame(additional_data)

# 利用抵達機場代號映射對應地區
data["Region"] = data["抵達機場代號"].map(airport_to_region)
data = data.merge(region_df, how="left", on="Region")

# --------------------------
# 5. 新增是否為平日欄位
# --------------------------
weekend_days = ["週六", "週日"]
data["是否為平日"] = data["星期"].apply(lambda x: 1 if x in weekend_days else 0)

# 將「是否為平日」移至「是否過夜」欄位之後
cols = data.columns.tolist()
if '是否過夜' in cols and '是否為平日' in cols:
    idx = cols.index("是否過夜") + 1
    cols.insert(idx, cols.pop(cols.index("是否為平日")))
data = data[cols]

# --------------------------
# 6. 新增機場客運量分類欄位
# --------------------------
super_large_airports = ['LHR', 'FRA', 'LAX', 'CDG', 'JFK']
medium_large_airports = ['SYD', 'ZRH']

data['機場客運量分類'] = data['抵達機場代號'].apply(
    lambda x: 1 if x in super_large_airports else (2 if x in medium_large_airports else None)
)

# 將「機場客運量分類」移至「抵達機場代號」後
cols = data.columns.tolist()
if '抵達機場代號' in cols and '機場客運量分類' in cols:
    arrival_idx = cols.index('抵達機場代號')
    cols.insert(arrival_idx + 1, cols.pop(cols.index('機場客運量分類')))
data = data[cols]

# --------------------------
# 7. 新增「飛行時間_分鐘」與「停留時間_分鐘」欄位
# --------------------------
def convert_time_to_minutes(time_str):
    """
    將格式如 'x 小時 y 分鐘' 轉換為分鐘數。
    若只有小時部分，也能正確解析。
    """
    if isinstance(time_str, str):
        match = re.match(r'(\d+)\s*小時(?:\s*(\d+)\s*分鐘)?', time_str)
        if match:
            hours = int(match.group(1))
            minutes = int(match.group(2)) if match.group(2) else 0
            return hours * 60 + minutes
    return None

# 新增飛行時間及停留時間轉換結果
data['飛行時間_分鐘'] = data['飛行時間'].apply(convert_time_to_minutes)
data['停留時間_分鐘'] = data['停留時間'].apply(convert_time_to_minutes)

data['飛行時間_分鐘'] = data['飛行時間_分鐘'].fillna(0).astype('int64')
data['停留時間_分鐘'] = data['停留時間_分鐘'].fillna(0).astype('int64')

# 將「飛行時間_分鐘」移至「飛行時間」後面
cols = data.columns.tolist()
if '飛行時間' in cols and '飛行時間_分鐘' in cols:
    flight_idx = cols.index('飛行時間')
    cols.insert(flight_idx + 1, cols.pop(cols.index('飛行時間_分鐘')))
    
# 將「停留時間_分鐘」移至「停留時間」後面
if '停留時間' in cols and '停留時間_分鐘' in cols:
    layover_idx = cols.index('停留時間')
    cols.insert(layover_idx + 1, cols.pop(cols.index('停留時間_分鐘')))
data = data[cols]

# --------------------------
# 8. 轉換航空公司組合及航空聯盟欄位型態
# --------------------------
if '航空公司組合' in data.columns:
    data['航空公司組合'] = data['航空公司組合'].astype('int64')
if '航空聯盟' in data.columns:
    data['航空聯盟'] = data['航空聯盟'].astype('int64')

# --------------------------
# 9. 匯出最終資料
# --------------------------
output_path = '/Users/yuchingchen/Documents/專題/cleaned_data/long_flight_final.csv'
data.to_csv(output_path, index=False)

抵達機場代號
LHR    9783
FRA    9168
LAX    8372
SYD    6758
CDG    5777
ZRH    5199
JFK    4560
Name: count, dtype: int64


In [5]:
# 顯示機型種類及數量
print(data['機型分類'].value_counts())

# 顯示 機型分類 = -1 的資料
print(data[data['機型分類'] == -1])

機型分類
 1    43916
 2     3011
 0     2172
-1      518
Name: count, dtype: int64
            出發日期     出發時間  出發時段   星期 出發機場代號                  抵達時間  抵達時段  \
1437  2025-02-04   清晨6:45  早晨班機   週二    TPE                晚上7:05  晚間抵達   
6783  2025-01-20   下午2:40  下午班機   週一    TSA   晚上7:00 (星期二, 1月 21)  晚間抵達   
6788  2025-01-20   下午4:30  下午班機   週一    TPE   晚上7:00 (星期二, 1月 21)  晚間抵達   
6808  2025-01-20  中午12:30  下午班機   週一    TSA   晚上7:00 (星期二, 1月 21)  晚間抵達   
6903  2025-01-21  中午12:30  下午班機   週二    TSA   晚上7:00 (星期三, 1月 22)  晚間抵達   
...          ...      ...   ...  ...    ...                   ...   ...   
45191 2025-02-22   晚上7:30  晚間班機  星期六    TPE   清晨7:00 (星期日, 2月 23)  早晨抵達   
45194 2025-02-22   晚上7:40  晚間班機  星期六    TPE   清晨7:00 (星期日, 2月 23)  早晨抵達   
45196 2025-02-22   晚上8:15  晚間班機  星期六    TPE   清晨7:00 (星期日, 2月 23)  早晨抵達   
45511 2025-02-24  晚上11:25  晚間班機  星期一    TPE  上午10:25 (星期二, 2月 25)  上午抵達   
45512 2025-02-24  晚上11:25  晚間班機  星期一    TPE   下午1:00 (星期二, 2月 25)  下午抵達   

      抵達機場代號  機場客運量分

# 長程航班-商務艙

In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

# --------------------------
# 1. 載入資料及初步處理
# --------------------------
data_path = '/Users/yuchingchen/Documents/專題/cleaned_data/long_flight_business.csv'
data = pd.read_csv(data_path, low_memory=False)

# 統一「出發日期」格式為 "YYYY-MM-DD"
data["出發日期"] = pd.to_datetime(
    data["出發日期"], 
    format="%Y-%m-%d", 
    errors="coerce"
).combine_first(
    pd.to_datetime(data["出發日期"], format="%Y/%m/%d", errors="coerce")
)
data["出發日期"] = data["出發日期"].dt.strftime("%Y-%m-%d")

# --------------------------
# 2. 計算對數變數與新增最低價格天數
# --------------------------
data["平均價格_log"] = data["平均價格"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan)
data["最低價格_log"] = data["最低價格"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan)
data["價格變異_log"] = data["價格變異"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan)
data["中位數價格_log"] = data["中位數價格"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan)

# 新增最低價格天數 (91 - '最低價格剩餘天數')
data['最低價格天數'] = 91 - data['最低價格剩餘天數']

# 移除關鍵欄位有 NaN 的資料
data = data.dropna(subset=["平均價格_log", "最低價格_log", "最低價格天數", "價格變異_log", "中位數價格_log"])

# 顯示抵達機場代號種類及數量
print(data['抵達機場代號'].value_counts())

# --------------------------
# 3. 假期判斷
# --------------------------
# 定義各國假期範圍
holidays = {
    "台灣": [("2025-01-25", "2025-02-02"), ("2025-02-28", "2025-03-02")],
    "美國": [("2025-01-18", "2025-01-20"), ("2025-02-15", "2025-02-17")],
    "澳洲": [("2025-01-25", "2025-01-27")],
    "瑞士": [("2025-01-29", "2025-01-30")]
}

# 抵達機場代號與地區對應
airport_to_region = {
    "LAX": "美國",  # 洛杉磯
    "SYD": "澳洲",  # 雪梨
    "FRA": "德國",  # 法蘭克福
    "CDG": "法國",  # 巴黎
    "JFK": "美國",  # 紐約
    "LHR": "英國",  # 倫敦
    "ZRH": "瑞士"   # 蘇黎世
}

# 轉換「出發日期」為 datetime 格式（方便比較）
data["出發日期"] = pd.to_datetime(data["出發日期"])

def is_holiday(row):
    # 台灣假期（不論目的地，出發地為台灣）
    for start, end in holidays["台灣"]:
        if pd.to_datetime(start) <= row["出發日期"] <= pd.to_datetime(end):
            return 1
    # 依據抵達機場對應的地區判斷其他國家假期
    region = airport_to_region.get(row["抵達機場代號"])
    if region and region in holidays:
        for start, end in holidays[region]:
            if pd.to_datetime(start) <= row["出發日期"] <= pd.to_datetime(end):
                return 1
    return 0

data["假期"] = data.apply(is_holiday, axis=1)

# 將「假期」欄位移至「艙等」欄位之後（若存在「艙等」欄位）
cols = data.columns.tolist()
if '假期' in cols and '艙等' in cols:
    cols.insert(cols.index('艙等') + 1, cols.pop(cols.index('假期')))
data = data[cols]

# --------------------------
# 4. 合併額外地區經濟資料
# --------------------------
additional_data = {
    "Region": ["美國", "澳洲", "德國", "英國", "法國", "瑞士"],
    "Cost of Living Index": [70.4, 70.2, 62.2, 62.0, 63.7, 101.1],
    "GDP (PPP) per capita (in thousand USD)": [82.715, 67.901, 69.532, 60.735, 63.881, 93.055],
}
region_df = pd.DataFrame(additional_data)

# 利用抵達機場代號映射對應地區
data["Region"] = data["抵達機場代號"].map(airport_to_region)
data = data.merge(region_df, how="left", on="Region")

# --------------------------
# 5. 新增是否為平日欄位
# --------------------------
weekend_days = ["週六", "週日"]
data["是否為平日"] = data["星期"].apply(lambda x: 1 if x in weekend_days else 0)

# 將「是否為平日」移至「是否過夜」欄位之後
cols = data.columns.tolist()
if '是否過夜' in cols and '是否為平日' in cols:
    idx = cols.index("是否過夜") + 1
    cols.insert(idx, cols.pop(cols.index("是否為平日")))
data = data[cols]

# --------------------------
# 6. 新增機場客運量分類欄位
# --------------------------
super_large_airports = ['LHR', 'FRA', 'LAX', 'CDG', 'JFK']
medium_large_airports = ['SYD', 'ZRH']

data['機場客運量分類'] = data['抵達機場代號'].apply(
    lambda x: 1 if x in super_large_airports else (2 if x in medium_large_airports else None)
)

# 將「機場客運量分類」移至「抵達機場代號」後
cols = data.columns.tolist()
if '抵達機場代號' in cols and '機場客運量分類' in cols:
    arrival_idx = cols.index('抵達機場代號')
    cols.insert(arrival_idx + 1, cols.pop(cols.index('機場客運量分類')))
data = data[cols]

# --------------------------
# 7. 新增「飛行時間_分鐘」與「停留時間_分鐘」欄位
# --------------------------
def convert_time_to_minutes(time_str):
    """
    將格式如 'x 小時 y 分鐘' 轉換為分鐘數。
    若只有小時部分，也能正確解析。
    """
    if isinstance(time_str, str):
        match = re.match(r'(\d+)\s*小時(?:\s*(\d+)\s*分鐘)?', time_str)
        if match:
            hours = int(match.group(1))
            minutes = int(match.group(2)) if match.group(2) else 0
            return hours * 60 + minutes
    return None

# 新增飛行時間及停留時間轉換結果
data['飛行時間_分鐘'] = data['飛行時間'].apply(convert_time_to_minutes)
data['停留時間_分鐘'] = data['停留時間'].apply(convert_time_to_minutes)

data['飛行時間_分鐘'] = data['飛行時間_分鐘'].fillna(0).astype('int64')
data['停留時間_分鐘'] = data['停留時間_分鐘'].fillna(0).astype('int64')

# 將「飛行時間_分鐘」移至「飛行時間」後面
cols = data.columns.tolist()
if '飛行時間' in cols and '飛行時間_分鐘' in cols:
    flight_idx = cols.index('飛行時間')
    cols.insert(flight_idx + 1, cols.pop(cols.index('飛行時間_分鐘')))
    
# 將「停留時間_分鐘」移至「停留時間」後面
if '停留時間' in cols and '停留時間_分鐘' in cols:
    layover_idx = cols.index('停留時間')
    cols.insert(layover_idx + 1, cols.pop(cols.index('停留時間_分鐘')))
data = data[cols]

# --------------------------
# 8. 轉換航空公司組合及航空聯盟欄位型態
# --------------------------
if '航空公司組合' in data.columns:
    data['航空公司組合'] = data['航空公司組合'].astype('int64')
if '航空聯盟' in data.columns:
    data['航空聯盟'] = data['航空聯盟'].astype('int64')

# --------------------------
# 9. 匯出最終資料
# --------------------------
output_path = '/Users/yuchingchen/Documents/專題/cleaned_data/long_flight_business_final.csv'
data.to_csv(output_path, index=False)